In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.distributions import Normal
import torch.optim as optim
from RL_brain import QLearningTable
# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 已有数据（保持不变）
seed_value = 1234
np.random.seed(seed_value)
UNIT = 40
MAZE_H = 8
MAZE_W = 8
utilization_ratios_device = 0.1
utilization_ratios_server = 0.1
d_flop = 1.3 * 10**12
d_power = 20
car_flop = 1.3 * 10**12*0.5
car_power = 20
e_flop = 330 * 10**12
e_power = 450
d_fai = 5*10**-29
trans_v_up = 100*1024*1024/4 #553081138.4484484
trans_v_dn = 20*1024*1024/4
p_cm = 0.1
# nums_data = np.array([5, 3, 4, 7, 9])  # 客户端本地数据量
partition_point = [0, 1, 2, 3, 4, 5, 6]
num_img_UAV = 3
num_img_car = 1

device_load = [0.3468e9, 0.3519e9, 2.3408e9, 2.3409e9, 5.3791e9, 9.6951e9, 12.077e9]
server_load = [11.7321e9, 11.727e9, 9.7381e9, 9.738e9, 6.6998e9, 2.3838e9, 0.0019e9]
exchanged_data = [2359296, 2359296, 2359296, 2359296, 1179628, 589824, 294912]
# 转换为NumPy数组
np_partition = np.array(partition_point)
np_device = np.array(device_load)
np_server = np.array(server_load)
np_exchanged = np.array(exchanged_data)



Using device: cuda


In [2]:
# index_temp = 1
# avg_t = 0
# r_i_u = 0
# for temp_2 in range(5):
#     b_i = 20e6  # 信道带宽 (Hz)
#     P_i_cm = 0.2  # 车载设备传输功率 (W)
    
#     r = 1.5  # 路径损耗指数
    
#     d_i = np.array([1000.0, 800.0, 600.0, 400.0, 200.0])  # 到边缘服务器的距离 (m)
#     # 计算信道增益
#     h_i = 1 / (d_i**r)
#     h_0 = 0.5
#     noise_density_dBm = -174  # 噪声功率谱密度 (dBm/Hz)
#     # 将噪声功率谱密度从 dBm/Hz 转换为 W/Hz
#     noise_density_W = 10**(noise_density_dBm / 10) * 1e-3  # 转换为 W/Hz
    
#     # 计算噪声功率
#     noise_power = noise_density_W * b_i
    
#     D_i = exchanged_data[6]#####
#     D_i_bits_actual = (D_i / 2) * 2*2  # 每数据点占用 16 位，单位为比特
#     r_i_u += b_i * np.log2(1 + (P_i_cm * h_i[temp_2]) / noise_power)###计算平均值
    
#     t_i_u = 7*D_i_bits_actual / r_i_u####时间
#     print(t_i_u)
#     e_i_cm = P_i_cm * t_i_u###能耗

# print(r_i_u/5)

In [3]:
def cost_cal(num_data, server_power, partition_index):
    partial_device = np_device[partition_index]
    device_time = partial_device * num_data / (d_flop *utilization_ratios_device)

    partial_server = np_server[partition_index]
    server_time = partial_server * num_data / ( e_flop* utilization_ratios_server + 1e-8)
    # print(f"device_time is {device_time}, server_time is {server_time}, cal_time is {device_time+server_time}")

    feature = np_exchanged[partition_index]
    trans_t_up = feature / trans_v_up * num_data
    # print(f"device_time is {device_time}, server_time is {server_time}, cal_time is {device_time+server_time},trans_t_up is {trans_t_up}")
    energy_cal = ((partial_device * d_power) / d_flop + (
            partial_server * e_power * utilization_ratios_server) / e_flop) * num_data
    energy_trans = num_data * p_cm * trans_t_up
    energy = energy_cal + energy_trans
    # print(f"energy cal is{energy_cal}, trans is {energy_trans}")
    landa_trans = 1
    time_all = device_time + server_time + landa_trans * trans_t_up
    return time_all, energy

In [4]:
class environment(object):
    def __init__(self, F_step_size=1, F_max=6, F_min=0):
        super(environment, self).__init__()
        self.server_power_step_size = F_step_size
        self.server_power_max = F_max
        self.server_power_min = F_min
        #self.num_user = device_index_1#####第几号设备

        self.action_space = ['l', 'r', 'no']
        self.n_actions = len(self.action_space)
        # self.title('cut the net')

        # create grids
        # for c in range(0, MAZE_W * UNIT, UNIT):####宽width
        #     x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
        #     self.canvas.create_line(x0, y0, x1, y1)#######画垂直线，起点坐标 (c, 0) 和终点坐标（c，w）
        # for r in range(0, UNIT, UNIT):
        #     x0, y0, x1, y1 = 0, r, UNIT, r#####画横线，起点坐标 (0, r) 和终点坐标（h，r）
        #     self.canvas.create_line(x0, y0, x1, y1)

        # create origin
#         origin = np.array([20, 20])
#         self.rect = self.canvas.create_rectangle(
#             origin[0] - 15, origin[1] - 15,
#             origin[0] + 15, origin[1] + 15,
#             fill='red')

#         # pack all
#         self.canvas.pack()
    # def reset(self):#########重新开始
    #     self.update()
    #     time.sleep(0.5)
    #     self.canvas.delete(self.rect)
    #     origin = np.array([20, 20])
    #     self.rect = self.canvas.create_rectangle(
    #         origin[0] - 15, origin[1] - 15,
    #         origin[0] + 15, origin[1] + 15,
    #         fill='red')
    #     return self.canvas.coords(self.rect)

    def step(self, state, action, num_img):
        
        state_next = 0
        # s = self.canvas.coords(self.rect)
        # base_action = np.array([0, 0])
        #print(action)

        if action == 1:   # 右边
            if state < 6:
                
                state_next = state+1
                   # record_action = 5
        elif action == 0:   # 左
            if state>0:
                state_next = state -1
        elif action == 2:###no move
            state_next = state
        #print(base_action[0])
        setpoint = state_next
        
            #print(0)
        time, energy = cost_cal(num_img, e_power, setpoint)
        # time_car, energy_car = cost_cal(num_img_car, e_power, action_car)
            # total_time += time
        reward = -time*0.4 - (energy)*0.3
        

        done = False#####不停，后期加停止命令
        
        return state_next, reward, done

In [5]:
# # index_1 = 0
# # num_data_temp = nums_data[index_1]###选着对应设备的对应训练个数nums_data = np.array([54, 30, 40, 91, 92])  # 客户端本地数据量
# #         #print(device_index_1)
# # server_power_temp = servers_power[index_1]
# # cost_time_temp , cost_energy_temp = cost_cal(num_data_temp, server_power_temp)
# servers_power_temp = [0.2, 0.2, 0.2, 0.2, 0.2]
# reward = 0
# for setpoint in range(5):
    
#     num_data_temp = nums_data[3]###选着对应设备的对应训练个数nums_data = np.array([54, 30, 40, 91, 92])  # 客户端本地数据量
#             #print(device_index_1)
#     server_power_temp = servers_power_temp[setpoint]
#     cost_time_temp , cost_energy_temp = cost_cal(1, server_power_temp)
#     landa_1 = 1########计算时间的权重
#     landa_2 = landa_1
#     reward += -1*(landa_1*cost_time_temp[4])# + landa_2*cost_energy[setpoint])
 
# print(f"五个设备的推理总时间为{reward*5}")

In [6]:
import pandas as pd
def update2():
    #num_step = 1
    # global fun_0
    # fun_0 = 1
    # action_note = 1
    #updated_q_table = pd.DataFrame(RL.q_table)
    
    initial_qtable = pd.DataFrame(columns=[0, 1, 2], dtype=np.float64)
    qtables = [initial_qtable.copy() for _ in range(2)]
    reward_list = [0.0001]*2
    action_list = [0]*2
    num_imgs = [3, 1]
    for i in range(40):####一共就训练十轮
        
        device_index = i%2
        #print(device_index)
        #print(f"Starting training with Qtable{device_index + 1}")
        
        RL = QLearningTable(actions=list(range(env.n_actions)), q_table=qtables[device_index])
        
        # trained_qtable = train_with_qtable(rl)
        # qtables[i] = trained_qtable
        record_1 = 0
        observation = 0
        while True:
            #device_index_1 = device_index
            action = RL.choose_action(str(observation), 1)
                #print(action)
                #print(format(observation))
            observation_, reward, done = env.step(observation, action, num_imgs[device_index])
            RL.learn(str(observation), action, reward, str(observation_))
            
            observation = observation_
            # if record_1+1 % 2000== 0:
            #     print(reward)
                #print(device_index_1)
            record_1 += 1    
            
            if record_1 % 400 == 0:
                if device_index == 1:
                    all_cost = sum(reward_list)#前几轮不要看
                    Q_history.append(all_cost)
            
                    print(f"reward is {all_cost}")
            if record_1 > 1000:
                reward_list[device_index] = abs(reward)
                
                trained_qtable = RL.q_table
                qtables[device_index] = trained_qtable
                
                
                #print(f"device{device_index+1} is {reward_list[device_index]}")
                break
        ###每五次都要输出一次总延时
        
    
Q_history = []
env = environment()
# print("env.n_actions: {}".format(env.n_actions))
#RL = QLearningTable(actions=list(range(env.n_actions)))


update2()
print(Q_history)

reward is 0.14238642832167833
reward is 0.14238642832167833
reward is 0.14238642832167833
reward is 0.14238642832167833
reward is 0.14238642832167833
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.22985755384615386
reward is 0.22985755384615386
reward is 0.22985755384615386
reward is 0.22985755384615386
reward is 0.22985755384615386
reward is 0.1841983568181818
reward is 0.1841983568181818
reward is 0.1841983568181818
reward is 0.1841983568181818
reward is 0.1841983568181818
reward is 0.23333768180068226
reward is 0.23333768180068226
reward is 0.23333768180068226
reward is 0.23333768180068226
reward is 0.23333768180068226
reward is 0.1843152377622378
reward is 0.1843152377622378
reward is 0.1843152377622378
reward is 0.1843152377622378
reward is 0.1843152377622378
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.18427627744755248
reward is 0.18427627

In [7]:
#0.01432499065792089, 0.010495516159809183, 0.059829394617885244, 0.02340235751373741, 0.007947835499900616]
print(device_index)

NameError: name 'device_index' is not defined

In [ ]:
print(Q_history)

In [ ]:
# servers_power = [0.4, 0.1, 0.15, 0.15, 0.2]
# device_index = 0
def update():#####这是多步的Qlearning
    #global fun_1
    
    num_step = 3
    gamma = 0.5  # reward_decay=0.5
    # 初始化记录变量
    global device_index
    global servers_power
    record_1 = 0
    sum_reward = [0] * num_step
    action_note = False
    servers_power = [0.4, 0.1, 0.15, 0.15, 0.2]
    initial_qtable_1 = pd.DataFrame(columns=[0, 1, 2], dtype=np.float64)
    qtables_1 = [initial_qtable_1.copy() for _ in range(5)]
    reward_list_1 = [0.0001]*5
    action_list = [0]*5
    for i in range(100):####一共就训练十轮
        
        
        device_index = i%5
        # print(f"Starting training with Qtable{device_index + 1}")
        
        
        RL = QLearningTable(actions=list(range(env.n_actions)), q_table=qtables_1[device_index])
        
        record_1 = 0
        observation = env.reset()
        
        while True:
            #device_index_1 = device_index
            observation_0 = observation

            for num in range(num_step):
                # 选择并执行动作
                action = RL.choose_action(str(observation), action_note)
                #env.render()
                observation_, reward, done = env.step(action)
    
                    # 更新奖励计算
                sum_reward[num] = reward + (gamma * sum_reward[num-1] if num > 0 else 0)
                observation = observation_
    
                    # 学习过程
                #print(f"Current reward: {reward}")
                RL.learn(str(observation_0), action, 1*sum_reward[num], str(observation_))
                record_1 += 1
                if record_1 >1500 :
                    
                    num_step = max(3, num_step-2) if num_step >= 3 else num_step
                    action_note = True
                # if record_1 % 2000 == 0:
                #     print(f"Current time cost: {reward}")
            if record_1 > 2000:
                reward_list_1[device_index] = abs(reward)
                action_list[device_index] = int(observation[0]/40)
                trained_qtable = RL.q_table
                qtables_1[device_index] = trained_qtable
                #print(f"device {device_index_1+1} cost is: {reward}")
                
                break
            
        ###每五次都要输出一次总延时
        print(f"Final position: {int(observation[0]/40)}")
        action = find_most_frequent(action_list)
        if device_index == 4:
            all_cost_1 = sum(reward_list_1)
            pro_Q_history.append(all_cost_1)
            print(all_cost_1)
            # 执行调整算力
            new_servers_power_1 = adjust_servers_power(servers_power, action, nums_data)
            
            servers_power = new_servers_power_1
            print(f"五个设备消耗为{reward_list_1}")
            print(f"服务器对设备分配算力为{servers_power}")
    #del device_index_1
    fun_0 = 0
    env.destroy()
    
fun_2 = 0 
cost_user = []*5    
pro_Q_history = []
#global device_index_1
env = environment()
# print("env.n_actions: {}".format(env.n_actions))
#RL = QLearningTable(actions=list(range(env.n_actions)))

env.after(100, update)
env.mainloop()
# updata2()
print(pro_Q_history)

In [ ]:
#%run multi_Qlearning.ipynb


In [ ]:
# %run run_this_DQN.py
Q_history = [0.3979611041451284, 0.5084312459310265, 0.45997046104223294, 0.4155057514597751, 0.459547224099518, 0.4462378191065669, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756]
pro_Q_history = [0.3968158409396685, 0.4617453188070799, 0.37802200520249274, 0.44907572209904656, 0.4253692783193946, 0.373271094649017, 0.35262858285274096, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756]
DQN_history_1 = [0.48700546107656695, 0.4611621587522712, 0.35139747398790977, 0.42886654008095887, 0.3894838051213463, 0.3230214205953048, 0.34388482811635274, 0.3494891499487529, 0.4029318371657479, 0.35885990446950455, 0.32245772397272465, 0.4271238647317376, 0.3223334323483416, 0.3222856011367443, 0.4156821318919513, 0.47961136324408704, 0.47453006798650277, 0.43264464520559187, 0.38520215257316476, 0.322121430158435]
multi_NET_history = [0.4147439057662382, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756, 0.3250607373578756]
step3_Q_history = [0.45939951965947967, 0.49532890232914795, 0.4593907102875944, 0.49418325389430184, 0.4342156428077455, 0.47603676515763577, 0.4580774311498178, 0.46602833710329405, 0.4771788679887338, 0.44704993493031153, 0.4615117340782279, 0.4949447807301173, 0.4906639463985181, 0.5587528338537002, 0.4435902799018353, 0.4995112137125783, 0.5238251378023414, 0.4647846076181629, 0.49834294905305476, 0.5038689740782576]
step5_Q_history = [0.49125958703019246, 0.43426354849160065, 0.4799999312991409, 0.44581651997374083, 0.5041475705226472, 0.4778233877080914, 0.4526234586215993, 0.500528963498544, 0.49725438862855054, 0.4952065676256895, 0.47263491436082666, 0.46274462002856187, 0.46804591958394004, 0.4748613268615772, 0.4879815723694093, 0.4456494800301122, 0.46331682965801035, 0.45936754510158695, 0.5044860959038258, 0.4633104089119022]

In [ ]:
x = list(range(len(pro_Q_history)))
y = Q_history
y_pro = pro_Q_history
plt.figure(figsize=(10, 6))
import matplotlib as mpl
# 统一长度，取最短的长度
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 20

# 假设x是各模型对应的epoch列表（这里假设每个历史列表长度相同，实际需根据数据调整）
# 定义函数获取每个列表的最小值及其索引
def get_min_point(history_list):
    min_value = min(history_list)
    min_index = history_list.index(min_value)
    return min_index, min_value

# 假设各历史列表：y, y_pro, step5_Q_history, step3_Q_history, DQN_history_1, multi_NET_history
# 分别获取每个列表的最小点
q_learning_min = get_min_point(y)
dynamic_multi_step_min = get_min_point(y_pro)
step5_q_min = get_min_point(step5_Q_history)
step3_q_min = get_min_point(step3_Q_history)
dqn_min = get_min_point(DQN_history_1)
multi_net_min = get_min_point(multi_NET_history)

# 绘制散点图

plt.scatter(q_learning_min[0], q_learning_min[1], marker='o', color='#fcf1f0', s=200, edgecolors='black', linewidths=0.8, label='Q - learning')
plt.scatter(dynamic_multi_step_min[0], dynamic_multi_step_min[1], marker='s', color='#fcccbc', edgecolors='black', linewidths=0.8, label='dynamic multi - step Q - learning')
plt.scatter(step5_q_min[0], step5_q_min[1], marker='^', color='#bdb5e1', s=200, edgecolors='black', linewidths=0.8, label='5 - step Q - learning')
plt.scatter(step3_q_min[0], step3_q_min[1], marker='d', color='#b0d992', s=200, edgecolors='black', linewidths=0.8, label='3 - step Q - learning')
plt.scatter(dqn_min[0], dqn_min[1], marker='*', color='#f9d580', s=200, edgecolors='black', linewidths=0.8, label='DQN')
plt.scatter(multi_net_min[0], multi_net_min[1], marker='*', color='#99b9e9', s=200, edgecolors='black', linewidths=0.8, label='our algorithm')
# 调整轴标签字体大小
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Time Consumption', fontsize=20)
plt.grid(True, linestyle='--', alpha=0.5)
# 调整图例字体大小
plt.legend(fontsize=16)
plt.show()

In [ ]:
#plt.savefig('G:/XYB/globecom/RL.png')  
plt.savefig('G:/XYB/globecom/picture/RL.png', dpi=300, bbox_inches='tight')  
plt.show()  

In [ ]:
x = list(range(len(pro_Q_history)))
y = Q_history
y_pro = pro_Q_history
plt.figure(figsize=(20, 16))
# 设置字体为Times New Roman并放大
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 20  # 可以根据需要调整大小

# 假设x是epoch列表，y等是各模型的Time consumption列表
x = range(len(multi_NET_history))  # 假设multi_NET_history是该模型的时间消耗列表

# 绘制散点图
# 绘制散点图
plt.scatter(x, y, marker='o', color='#fcf1f0', label='Q - learning')
plt.scatter(x, y_pro, marker='s', color='#fcccbc', label='dynamic multi - step Q - learning')
plt.scatter(x, step5_Q_history, marker='^', color='#bdb5e1', label='5 - step Q - learning')
plt.scatter(x, step3_Q_history, marker='d', color='#b0d992', label='3 - step Q - learning')
plt.scatter(x, DQN_history_1, marker='*', color='#f9d580', label='DQN')
plt.scatter(x, multi_NET_history, marker='*', color='#99b9e9', s=160, label='our algorithm')

# 绘制平滑连接曲线
plt.plot(x, y, color='#fcf1f0', linestyle='--', alpha=0.7, label='Q - learning')
plt.plot(x, y_pro, color='#fcccbc', linestyle='--', alpha=0.7, label='dynamic multi - step Q - learning')
plt.plot(x, step5_Q_history, color='#bdb5e1', linestyle='--', alpha=0.7, label='5 - step Q - learning')
plt.plot(x, step3_Q_history, color='#b0d992', linestyle='--', alpha=0.7, label='3 - step Q - learning')
plt.plot(x, DQN_history_1, color='#f9d580', linestyle='--', alpha=0.7, label='DQN')
plt.plot(x, multi_NET_history, color='#99b9e9', linestyle='-', linewidth=2, alpha=0.8, label='our algorithm')

plt.xlabel('Epoch to Reach Minimum Time Consumption', fontsize=20)  # 调整标签和字体大小
plt.ylabel('Time Consumption', fontsize=20)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(fontsize=16)  # 调整图例字体大小
plt.show()